# Prelims

In [1]:
import math
import pandas as pd
import random as rd

from tqdm.notebook import tqdm
from typing import List, Tuple, Union
from uuid import uuid4

# Parameters

In [2]:
# General parameters
PRICE_MIN,PRICE_MAX = 1, 100
QTY_MIN, QTY_MAX = 0, 100

In [3]:
# Sellers' parameters
NB_SELLERS = 5
PRICE_PROD = 30

MEMORY_SELLER = 0.5
RISK_AVERSITY_SELLER = 0.2
CURIOSITY_SELLER = 0.3

In [4]:
# Buyers' parameters
NB_BUYERS = 5
BUDGET = 70

MEMORY_BUYER = 0.5
RISK_AVERSITY_BUYER = 0.2
CURIOSITY_BUYER = 0.3
MYOPIA_FACTOR = 0.7
PENALTY_FACTOR = 1/PRICE_PROD   # 0.5

# Agents definition

In [5]:
class Seller:
    """
        Selling agent class
        * name: Unique identifier 
        * alpha: Q-learning alpha factor, representing agent's memory
        * gamma: Q-learning gamma factor, representing agent's risk aversity
        * epsilon: e-greedy policy e factor, representing agent's curiosity
        * price_prod: Price of producing one unit of good   # TODO: Replace to price function
        * qty_prod: Quantity of goods produced (to be learned as we go along)
        * price_sell: Price at which the agent will try to sell its goods (to be learned as we go along)
        * qty_left: Quantity of goods that have not been sold (yet)
        * q_table: Q-learning table
        * history: List of all sales
    """
    
    def __init__(self,  
                 price_prod: int = PRICE_PROD, 
                 alpha: float = MEMORY_SELLER, 
                 gamma: float = RISK_AVERSITY_SELLER, 
                 epsilon: float = CURIOSITY_SELLER,
                 stochastic: bool = False,
                 name: Union[str, int] = ""):
        
        self.name: str = str(name) if name else str(uuid4())
        self.alpha: float = alpha
        self.gamma: float = gamma
        self.epsilon: float = epsilon
        
        assert PRICE_MIN <= price_prod <= PRICE_MAX, f"Production price price_prod={price_prod} is not within the price bounds PRICE_MIN={PRICE_MIN} and PRICE_MAX={PRICE_MAX}"
        self.price_prod: int = price_prod
        
        # If stochastic==True, use a gaussian distribution to get price_prod
        # TODO: Try out other distributions
        if stochastic:
            std_dev = (PRICE_MAX - PRICE_MIN) / 100
            price_prod = rd.gauss(price_prod, std_dev)
            self.price_prod = min(PRICE_MAX, max(PRICE_MIN, floor(price_prod)))
            
        # Initialize randomly first selling price
        # TODO: Try out other initialization strategies
        self.qty_prod: int = rd.randint(self.price_prod, PRICE_MAX)
        self.price_sell: int = rd.randint(self.price_prod, PRICE_MAX)
        
        self.qty_left: int = self.qty_prod
        
        # Initialize Q-table with zeros
        self.q_table: pd.DataFrame = pd.DataFrame(
            index=range(PRICE_MIN, PRICE_MAX+1), 
            columns=range(QTY_MIN, QTY_MAX+1), 
            dtype=float
        ).fillna(0.)
            
        # history is a list of lists (one for every round) of pairs (price, quantity)
        self.history: List[List[Tuple[int, int]]] = [[]]
            
            
    def get_price(self) -> int:
        return self.price_sell
            
    def get_qty(self) -> int:
        return self.qty_left
            
    def get_history(self) -> list:
        return self.history
    
    
    def sell(self, qty: int) -> None:
        """ Sell qty goods at self.price_sell """
        assert qty <= self.qty_left, f"Seller {seller.name} is trying to sell {qty} goods, but it has only {self.qty_left} left!"
        self.qty_left -= qty
        self.history[-1].append((self.price_sell, qty))
        
            
    def learn(self) -> None:
        """ Update Q-table ("learn") and reinitialize params """
        
        # Compute reward (profit)
        qty_sold = self.qty_prod - self.qty_left
        reward = qty_sold * self.price_sell - self.qty_prod * self.price_prod
        
        # Update Q-table
        self.q_table.loc[self.price_sell, self.qty_prod] *= 1 - self.alpha
        self.q_table.loc[self.price_sell, self.qty_prod] += self.alpha * reward   # + self.alpha * self.gamma * self.q_table.max()
        
        # Get next price with e-greedy policy
        # TODO: Try out other exploration/exploitation policies
        if rd.random() < self.epsilon:
            # Exploration: Try out a random pair (price_sell, qty_prod)
            self.price_sell = rd.choice(list(self.q_table.index))
            self.qty_prod = rd.choice(list(self.q_table.columns))
        else:
            # Exploitation: Go for maximizing pair (price_sell, qty_prod)
            self.price_sell = self.q_table.index[self.q_table.max(axis=1).argmax()]
            self.qty_prod = self.q_table.columns[self.q_table.max().argmax()]
        
        # Prepare a new list for next round
        self.history.append([])

In [6]:
class Buyer:
    """
        Buying agent class
        * name: Unique identifier 
        * alpha: Q-learning alpha factor, representing agent's memory
        * gamma: Q-learning gamma factor, representing agent's risk aversity
        * epsilon: e-greedy policy e factor, representing agent's curiosity
        * myopia: Factor representing agent's short-sightedness
        * penalty: Penalization factor when there are some budget leftovers at the end of the round
        * budget: Initial budget when starting round
        * budget_left: Budget left, to be used
        * q_table: Q-learning table
        * history: List of all purchases
    """
    
    def __init__(self,  
                 budget: int = BUDGET, 
                 alpha: float = MEMORY_BUYER, 
                 gamma: float = RISK_AVERSITY_BUYER, 
                 epsilon: float = CURIOSITY_BUYER,
                 myopia: float = MYOPIA_FACTOR,
                 penalty: float = PENALTY_FACTOR,
                 stochastic: bool = False,
                 name: Union[str, int] = ""):
        
        self.name: str = str(name) if name else str(uuid4())
        self.alpha: float = alpha
        self.gamma: float = gamma
        self.epsilon: float = epsilon
        self.myopia: float = myopia
        self.penalty: float = penalty
        
        assert PRICE_MIN <= budget <= PRICE_MAX, f"Budget budget={budget} is not within the price bounds PRICE_MIN={PRICE_MIN} and PRICE_MAX={PRICE_MAX}"
        self.budget: int = budget
        
        # If stochastic==True, use a gaussian distribution to get budget
        # TODO: Try out other distributions
        if stochastic:
            std_dev = (PRICE_MAX - PRICE_MIN) / 100
            budget = rd.gauss(budget, std_dev)
            self.budget = min(PRICE_MAX, max(PRICE_MIN, floor(budget)))
            
        self.budget_left = self.budget
        
        # Initialize Q-table with zeros
        index_list = [ (i, j) for i in range(PRICE_MIN, PRICE_MAX+1) for j in range(PRICE_MIN, PRICE_MAX+1) ]
        self.q_table: pd.DataFrame = pd.DataFrame(
            index=pd.MultiIndex.from_tuples(index_list), 
            columns=range(QTY_MIN, QTY_MAX+1), 
            dtype=float
        ).fillna(0.)
            
        # history is a list of lists (one for every round) of triples (budget_before, price, quantity)
        self.history: List[List[Tuple[int, int, int]]] = [[]]
            
            
    def get_history(self) -> list:
        return self.history
    
    
    def buy(self, price: int, qty_left: int) -> int:
        """ Returns the number of goods bought at price price """
        
        # Get quantity to buy with e-greedy policy, given that
        # * quantity shall be lower than qty_left (Buyer cannot buy more goods than available)
        # * quantity * price shall be lower than budget (Buyer cannot buy goods for more than its budget)
        # TODO: Try out other exploration/exploitation policies
        
        qty_max = min(qty_left, self.budget // price)
        if rd.random() < self.epsilon:
            # Exploration: Try out a random quantity
            qty_to_buy = rd.choice(list(self.q_table.columns[:qty_max+1]))
        else:
            # Exploitation: Go for maximizing quantity
            qty_to_buy = self.q_table.columns[self.q_table.max()[:qty_max+1].argmax()] 
        
        self.history[-1].append(( self.budget_left, price, qty_to_buy ))
        self.budget_left -= qty_to_buy * price
        return qty_to_buy
        
            
    def learn(self) -> None:
        """ Update Q-table ("learn") and reinitialize params """
        
        # Compute number of goods purchased overall this round
        nb_goods_purchased_after = sum([ qty for _, _, qty in self.history[-1] ])
        nb_purchases = len(self.history[-1])
        
        # Compute penalty for budget not spent
        penalty = self.penalty * self.budget_left
        
        for i, purchase in enumerate(self.history[-1]):
            budget, price, qty = purchase
            
            # Compute reward (nb of goods purchased)
            # Weight more nb of goods purchased 
            myopia_factor = self.myopia**(nb_purchases - i)
            nb_goods_purchased_after -= qty
            reward = qty + myopia_factor * (nb_goods_purchased_after - penalty)
            
            # Get max potential Q-value
            if i < nb_purchases - 1:
                budget_left, _, _ = self.history[-1][i+1]
                potential_reward = self.q_table.loc[(budget_left, price)].max()   # Pb if qty=0, it may be an already updated value (really pbic?)
            else:
                potential_reward = 0
        
            # Update Q-table
            self.q_table.loc[(budget, price), qty] *= 1 - self.alpha
            self.q_table.loc[(budget, price), qty] += self.alpha * (reward + self.gamma * potential_reward)
        
        # Prepare a new list for next round
        self.history.append([])

# Game definition

In [7]:
def get_new_agents(nb_sellers: int = NB_SELLERS, nb_buyers: int = NB_BUYERS) -> Tuple[List[Seller], List[Buyer]]:
    """ Creates lists of sellers/buyers """
    sellers = [ 
        Seller(
            name=i,
            price_prod=30
        )
        for i in range(nb_sellers) 
    ]
    buyers = [ 
        Buyer(
            name=i,
            budget=70
        )
        for i in range(nb_buyers) 
    ]
    return sellers, buyers

In [8]:
def play_round(sellers: List[Seller], buyers: List[Buyer], Verbose: bool = True) -> Tuple[List[Seller], List[Buyer]]:
    """ Plays a round """
    
    # Consider buyers in a random order
    rd.shuffle(buyers)

    for buyer in buyers:
        if Verbose:
            print(f"> Buyer {buyer.name}: budget_left={buyer.budget_left}")
        
        # Get a random number of sellers to present to the buyer
        # TODO: Try out other strategies
        k = rd.randint(0, NB_SELLERS)
        idx_sellers = rd.sample(range(NB_SELLERS), k=k)
        
        for idx_seller in idx_sellers:
            
            seller = sellers[idx_seller]
            price, qty = seller.get_price(), seller.get_qty()
            if Verbose:
                print(f">> Seller {seller.name}: price_sell={price}  qty_left={qty}")

            qty_bought = buyer.buy(price, qty)
            sellers[idx_seller].sell(qty_bought)
            if Verbose:
                print(f">>> {qty_bought} goods purchased/sold")
                print(f">>> Buyer's budget_left={buyer.budget_left}")
                print(f">>> Seller's qty_left={sellers[idx_seller].get_qty()}")
                
        buyer.learn()
                    
    for seller in sellers:
        seller.learn()
        
    return sellers, buyers

In [9]:
# def display_history(sellers: list):
#     history = pd.DataFrame({
#         seller.id: seller.get_history()
#         for seller in sellers
#     })
#     history.plot()

# Plays

In [10]:
sellers, buyers = get_new_agents()

for _ in tqdm(range(10)):
    sellers, buyers = play_round(sellers, buyers, Verbose=True)
    
# display_history(sellers)

  0%|          | 0/10 [00:00<?, ?it/s]

> Buyer 6e7ffcac-af71-4368-9d81-08eadc99ea39: budget_left=70
>> Seller 2: price_sell=69  qty_left=86
>>> 0 goods purchased/sold
>>> Buyer's budget_left=70
>>> Seller's qty_left=86
>> Seller 0b013a21-bfae-4ac7-8606-2d98ccc1f3d0: price_sell=73  qty_left=78
>>> 0 goods purchased/sold
>>> Buyer's budget_left=70
>>> Seller's qty_left=78
>> Seller 1: price_sell=64  qty_left=62
>>> 0 goods purchased/sold
>>> Buyer's budget_left=70
>>> Seller's qty_left=62
>> Seller 3: price_sell=46  qty_left=45
>>> 0 goods purchased/sold
>>> Buyer's budget_left=70
>>> Seller's qty_left=45
>> Seller 4: price_sell=63  qty_left=64
>>> 1 goods purchased/sold
>>> Buyer's budget_left=7
>>> Seller's qty_left=63
> Buyer 3: budget_left=70
>> Seller 3: price_sell=46  qty_left=45
>>> 0 goods purchased/sold
>>> Buyer's budget_left=70
>>> Seller's qty_left=45
>> Seller 4: price_sell=63  qty_left=63
>>> 1 goods purchased/sold
>>> Buyer's budget_left=7
>>> Seller's qty_left=62
>> Seller 2: price_sell=69  qty_left=86
>>> 0 g

KeyError: -62